# Assignment 3 Template


In [1]:
import numpy as np
import pandas as pd
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler

#### 1. Split Data and Perform Scaling

In [2]:
# Load dataset
#Import Auto.csv
#Split into train and test sets
#Apply scaling (StandardScaler)

df = pd.read_csv('Auto.csv')
auto_df = pd.read_csv('Auto.csv')
display(auto_df)
(auto_df=='?').sum()

import statistics
auto_df_ = auto_df[auto_df['horsepower'] != '?']
(auto_df_=='?').sum()
auto_df_ = auto_df_.drop(columns=['name'])
auto_df_
mpg_median = statistics.median(auto_df_['mpg'].to_list())
auto_df_.iloc[auto_df_['mpg']<mpg_median,0] = 0
auto_df_.iloc[auto_df_['mpg']>=mpg_median,0] = 1
auto_df_['mpg']=auto_df_['mpg'].astype(int)
auto_df_

from sklearn.model_selection import train_test_split
auto_x_train, auto_x_test, y_train, y_test = train_test_split(auto_df_.drop(columns=['mpg']),auto_df_['mpg'], test_size=0.20, random_state=42)
scaler = StandardScaler()
auto_x_train_scaled = scaler.fit_transform(auto_x_train)
auto_x_test_scaled =scaler.transform(auto_x_test)



,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,8,307.0,130,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140,3449,10.5,70,1,ford torino
...,...,...,...,...,...,...,...,...,...
392,27.0,4,140.0,86,2790,15.6,82,1,ford mustang gl
393,44.0,4,97.0,52,2130,24.6,82,2,vw pickup
394,32.0,4,135.0,84,2295,11.6,82,1,dodge rampage
395,28.0,4,120.0,79,2625,18.6,82,1,ford ranger


#### 2. Neural Network Model (MLP) with GridSearchCV

In [9]:
# Define parameter grid for hidden layer sizes
# Train MLPClassifier with GridSearchCV (5-fold)
# Evaluate accuracy on test set

from sklearn.neural_network import MLPClassifier

nn_model = MLPClassifier (hidden_layer_sizes=(20,2),# two hidden layers
                                                 max_iter=100,
                                                 activation = 'relu',
                                                 solver='adam',batch_size=200,# mini batch
                                                 n_iter_no_change=10,
                                                 early_stopping=True) #early stopping
nn_model.fit(auto_x_train_scaled,y_train)
nn_model.predict(auto_x_test_scaled)
print(accuracy_score(y_train,nn_model.predict(auto_x_train_scaled)))
print(accuracy_score(y_test,nn_model.predict(auto_x_test_scaled)))
from sklearn.model_selection import GridSearchCV

mlp_grid ={'hidden_layer_sizes':[(2),(5),(10),(15),(20),(50),(50,50),(100,100),(200,200),(100,100,100),(200,200,200),(500,500,500),(1000,1000,1000)]}
nn_model = MLPClassifier (max_iter=5000,
                           activation = 'relu',
                           solver='adam',batch_size=5,
                           n_iter_no_change=10,
                           early_stopping=False)
clf = GridSearchCV(nn_model, 
                   param_grid = mlp_grid,
                   scoring = 'accuracy',
                   cv = 5, 
                   n_jobs = 4,
                   return_train_score = True)
clf.fit(auto_x_train_scaled,y_train)
print("Accuracy",accuracy_score(y_test,y_pred))


0.49201277955271566
0.5316455696202531
Accuracy 0.9113924050632911


#### 3. Logistic Regression Model with GridSearchCV

In [5]:
# Define parameter grid for 'solver'
# Train LogisticRegression with GridSearchCV (5-fold)
# Evaluate accuracy on test set
# Inspect coefficients and comment if they make sense

log_model = LogisticRegression()
parm_grid = {'solver':['newton-cg','lbfgs','liblinear','sag','saga']}
clf = GridSearchCV(log_model, param_grid= parm_grid,
                   scoring='accuracy', cv=5,
                   n_jobs=4,
                   return_train_score=True)

clf.fit(auto_x_train_scaled,y_train)
print(clf.best_params_)
log_model = LogisticRegression(solver=clf.best_params_['solver'])

log_model.fit(auto_x_train_scaled,y_train)
y_pred = log_model.predict(auto_x_test_scaled)
print("Accuracy",accuracy_score(y_test,y_pred))
coeff_df = pd.DataFrame({'Feature':auto_x_train.columns,'Coefficient':log_model.coef_[0]})
print(coeff_df)


{'solver': 'liblinear'}
Accuracy 0.8481012658227848
        Feature  Coefficient
0     cylinders    -0.367073
1  displacement    -0.116181
2    horsepower    -1.374740
3        weight    -2.336532
4  acceleration    -0.024941
5          year     1.394063
6        origin     0.522145


###### comments on coefficients

Yes, these coefficients make sense because a negative coefficient means the log-odds decrease as the feature increases. So, as the number of cylinders, displacement, horsepower, weight, and acceleration increase, the probability of having good MPG decreases — in other words, the car is more likely to have poor MPG. On the other hand, year and origin have positive coefficients, which increase the probability of good MPG. This aligns with the idea that newer cars and cars from outside the USA typically have better fuel efficiency.

#### 4. K-Nearest Neighbors Model with GridSearchCV

In [7]:
# Define parameter grid for n_neighbors
# Train KNN with GridSearchCV (5-fold)
# Evaluate accuracy on test set
from sklearn.neighbors import KNeighborsClassifier

parm_grid = {'n_neighbors':[1,3,5,7,9,11,13,15,17,19,21,23,25]}
knn_model = KNeighborsClassifier()
clf = GridSearchCV(knn_model, param_grid= parm_grid,
                   scoring='accuracy', cv=5,
                   n_jobs=4,
                   return_train_score=True)

clf.fit(auto_x_train_scaled,y_train)
knn_model = KNeighborsClassifier(n_neighbors=clf.best_params_['n_neighbors'])
knn_model.fit(auto_x_train_scaled,y_train)
y_pred = knn_model.predict(auto_x_test_scaled)

print(clf.best_params_)
print("Accuracy",accuracy_score(y_test,y_pred))

{'n_neighbors': 7}
Accuracy 0.9113924050632911


#### 5. Comparison of Accuracies

###### Compare accuracies from sections 2, 3, and 4.
